# 1. 환경설정

In [ ]:
!pip install openai

In [ ]:
!pip show openai

Name: openai
Version: 1.57.4
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 


In [ ]:
import openai
from openai import OpenAI
import os
import json
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# OpenAI API 키 설정
openai.api_key = "..."

# 2. 데이터 준비

In [ ]:
# 데이터 로드
# 발화 분리, 감정이 태깅된 자유대화(성인) inside
file_path = '/content/drive/MyDrive/AIFFELthon/Data/Sample/대화별_발화분리_텍스트+감정_csv_(json에서_변환)/GPT_input/utterance_json_output_inside-GPT_input.csv'  # 업로드된 파일 경로
data = pd.read_csv(file_path)

# 필요한 열만 추출
data = data[['FileName', 'Text']]

In [ ]:
# 파일명 기준으로 대화 분리
def group_dialogues_by_filename(data):
    # 파일명 기준 그룹화
    data['DialogueGroup'] = data['FileName'].str.extract(r'(.+?)_\d{6}_\d{6}.wav')
    grouped_data = data.groupby('DialogueGroup')['Text'].apply(list).reset_index()
    return grouped_data

In [ ]:
# 대화 데이터 분리
grouped_data = group_dialogues_by_filename(data)
dialogues = grouped_data['Text'].tolist()

In [ ]:
dialogues[1][:10]

['아까 제가 시간 가는 줄도 모르고 계속 얘기를 했네요',
 '여보세요',
 '여보세요',
 '아닙니다',
 '잘 듣고 있었습니다',
 '저는',
 '모두가 그냥',
 '스키장에서 못 타는',
 '그럼 보두 그럼',
 '게임인 줄 알았는데']

# 3. GPT 레이블링
- api 호출
- 인지적 프롬프팅 : STICC

In [ ]:
def create_chat_completion(sentence, system_input, model="gpt-4o-mini", temperature=1.15, max_tokens=150):
    try:
        user_input = f"""
        문장: "{sentence}"
        위 문장을 분석하고 아래 JSON 스키마에 맞춰 결과를 출력해줘:
        ```json
        {{
          "name": "emotion_analysis",
          "strict": true,
          "schema": {{
            "type": "object",
            "required": [
              "primary_emotion",
              "secondary_emotion"
            ],
            "properties": {{
              "primary_emotion": {{
                "enum": [
                  "기쁨",
                  "놀라움",
                  "두려움",
                  "사랑스러움",
                  "슬픔",
                  "화남",
                  "없음"
                ],
                "type": "string",
                "description": "감정 분류 체계에 따라 선택된 가장 가능성 높은 감정."
              }},
              "secondary_emotion": {{
                "enum": [
                  "기쁨",
                  "놀라움",
                  "두려움",
                  "사랑스러움",
                  "슬픔",
                  "화남",
                  "없음"
                ],
                "type": "string",
                "description": "감정 분류 체계에 따라 선택된 두번째로 가능성 높은 감정."
              }}
            }},
            "additionalProperties": false
          }}
        }}
        ```
        """
        # 메시지 목록을 자동으로 생성
        messages = [
            {"role": "system", "content": system_input},
            {"role": "user", "content": user_input}
        ]

        response = OpenAI().chat.completions.create(  # return a JSON response
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens  # 최대 토큰 수를 지정
        )
        content = response.choices[0].message.content
        # return response
        clean_content = content.strip("```json").strip("```").strip()
        return json.loads(clean_content)
    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}")
        return {"error": "Invalid JSON format"}
    except Exception as e:
        print(f"Error: {str(e)}")
        return f"Error: {str(e)}"

# 메시지 목록 예시
system_input = """
**Situation**: 나는 자막에 감정 정보를 반영하는 기술 개발을 위해 text 데이터를 기반으로 감정 레이블링 작업을 하고 있어.

**Task**: 네가 문장별로 감정을 분석하고 레이블링을 도와줘. 감정 상태 레이블링 체계는 ‘기쁨,놀라움,두려움,사랑스러움,슬픔,화남,없음’으로 이루어져 있어. 감정 유형 체계는 ‘긍정, 부정, 중립’으로 이루어져 있어. 이 체계를 따라서 레이블링을 해줘. 그리고 감정 상태와 유형 각각 1순위, 2순위를 나누어서 레이블링 해줘.

**Intent**: 감정 분류 체계와 감정 유형을 정확히 반영해서 데이터의 품질을 높이는 게 목표야.

**Concern**: 텍스트의 문맥이 불분명한 경우 감정을 잘못 분류할까 걱정이야.

**Calibration**: 모호한 문장은 '없음'으로 레이블링하되, 가능한 한 분류 체계를 따르도록 노력해줘.
"""


# API 호출 및 결과 출력
# responses = create_chat_completion("너를 사랑해",system_input)
# print(responses)
# print(responses.choices[0].message.content)

In [ ]:
sample_dlg = dialogues[:1]

In [ ]:
print(sample_dlg[:5])  # 첫 5개의 문장 확인
print(type(sample_dlg[0]))  # 첫 번째 문장의 데이터 타입 확인

[['안녕하세요', '네 안녕하세요', '식사는 하셨습니까', '저기 밥을 먹었는지 어쨌는지 지금 모를 정도로', '아 지칩니다', '소리 조금만 켜주시겠어요', '아 제가 얻게 될 힘도 없어요 지금', '아 지금은 쪼끔', '들을 들릴만합니다', '바쁘셨나 봐요', '하는 거 혹시 왜 이렇게 지칠까요', '어 그러게요', '어 우리가 지금 주제가 올림픽인데', '씩씩해야 되는데', '벌써 지치면', '아마 얘기를 하다 보면 더 흥분이 되어서', '아 결혼', '음', '음', '아 목소리 톤이', '음', '올라가지 않을까 싶네요', '어', '휴일인데 많이 바쁘셨나 봐요', '제가 이렇게 바쁜 사람이 아니거든요', '아 그래요', '그런데', '가만히 보면', '네', '무슨 일이 하 어 일어날 때는 한꺼번에 이렇게 겹치는', '아 맞아요', '네', '그렇게 되더라구요 이게 본인 아니게 그렇게 되니까 제가 또 못 미', '빨아주지를 못해요 싹 한 개를 느낄 때가 많네요', '어', '맞아요', '저만 그런 줄 알았는데 다들 그러구나', '또 한가할 땐 너무 한가하고', '아닙니다', '그 동계올림픽', '으이 제가 주제를 가져온 이에 이유가 뭐냐면', '네', '곧 있으면 이제 겨울로 저거 들기 시작하는데', '그렇죠', '그러다 보면', '네 겨울 스포츠가 또 빠질 수는 없겠죠', '네', '으', '그 저희 나라에서', '동계올림픽을', '어 이천 십 팔 년도인가', '개최를 했잖아요', '어', '시작', '그때', '강원도 평창에서 했는데', '네', '그 개회식 때', '그 퍼포 먼스 중에 하나가', '네', '너무 가슴이 벅찰 정도로 감동적인', '그 장면이 있어서', '그걸 한 번 짚고 넘어가고 싶은데 혹시 기억이 나시는나 모르겠는데', '아', '보셨나요', '음 그', '개막식을', '보기는 했는데', '네', '어 그때 작년이 생생하게 떠오르진 않아요', '하지만', '어느 한 장면이 아직도 가슴에 뭉클하게 남는 장면이 있어요', '어떤

In [ ]:
# 결과 저장 리스트
results = []

for i, sentence in enumerate(sample_dlg[0]):
    print(f"Processing sentence {i + 1}/{len(sample_dlg[0])}")
    try:
        # 감정 레이블링 수행
        result = create_chat_completion(sentence, system_input)
        results.append({
            "sentence_id": i + 1,
            "sentence": sentence,
            "result": result  # 감정 레이블링 결과
        })
    except json.JSONDecodeError as e:
        # JSON 디코드 오류 처리
        print(f"JSON Decode Error: {e} for sentence: {sentence}")
        results.append({
            "sentence_id": i + 1,
            "sentence": sentence,
            "result": {"error": f"JSON Decode Error: {e}"}
        })
    except Exception as e:
        # 일반 오류 처리
        print(f"Error: {e} for sentence: {sentence}")
        results.append({
            "sentence_id": i + 1,
            "sentence": sentence,
            "result": {"error": str(e)}
        })

    # 현재 처리된 결과를 주기적으로 저장 (안전한 데이터 보존)
    if (i + 1) % 50 == 0:  # 50문장마다 중간 저장
        temp_output_file = f"temp_emotion_analysis_{i + 1}.json"
        with open(temp_output_file, "w", encoding="utf-8") as f:
            json.dump(results, f, ensure_ascii=False, indent=4)
        print(f"Temporary results saved to {temp_output_file}")

# 최종 결과를 JSON 파일로 저장
output_file = "emotion_analysis_sentences.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file}")

### 기존 저장된 것 이후부터 진행하는 코드

In [ ]:
# 기존에 저장된 결과 파일 읽기
results = []
processed_ids = set()  # 처리된 문장의 ID를 저장

output_file = "/content/temp_emotion_analysis_300.json"

if os.path.exists(output_file):
    with open(output_file, "r", encoding="utf-8") as f:
        results = json.load(f)
        processed_ids = {item["sentence_id"] for item in results}  # 이미 처리된 문장의 ID

print(f"Loaded {len(processed_ids)} sentences from previous results.")

# 301번부터 시작
for i, sentence in enumerate(sample_dlg[0]):
    sentence_id = i + 1
    if sentence_id in processed_ids:
        continue  # 이미 처리된 문장은 건너뜀

    print(f"Processing sentence {sentence_id}/{len(sample_dlg[0])}")
    try:
        # 감정 레이블링 수행
        result = create_chat_completion(sentence, system_input)
        results.append({
            "sentence_id": sentence_id,
            "sentence": sentence,
            "result": result  # 감정 레이블링 결과
        })
    except json.JSONDecodeError as e:
        # JSON 디코드 오류 처리
        print(f"JSON Decode Error: {e} for sentence: {sentence}")
        results.append({
            "sentence_id": sentence_id,
            "sentence": sentence,
            "result": {"error": f"JSON Decode Error: {e}"}
        })
    except Exception as e:
        # 일반 오류 처리
        print(f"Error: {e} for sentence: {sentence}")
        results.append({
            "sentence_id": sentence_id,
            "sentence": sentence,
            "result": {"error": str(e)}
        })

    # 현재 처리된 결과를 주기적으로 저장 (안전한 데이터 보존)
    if sentence_id % 50 == 0:  # 50문장마다 중간 저장
        temp_output_file = f"temp_emotion_analysis_{sentence_id}.json"
        with open(temp_output_file, "w", encoding="utf-8") as f:
            json.dump(results, f, ensure_ascii=False, indent=4)
        print(f"Temporary results saved to {temp_output_file}")

# 최종 결과를 JSON 파일로 저장
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file}")


Loaded 300 sentences from previous results.
Processing sentence 301/478
Processing sentence 302/478
Processing sentence 303/478
Processing sentence 304/478
Processing sentence 305/478
Processing sentence 306/478
Processing sentence 307/478
Processing sentence 308/478
Processing sentence 309/478
Processing sentence 310/478
Processing sentence 311/478
Processing sentence 312/478
Processing sentence 313/478
JSON Decode Error: Extra data: line 5 column 1 (char 59)
Processing sentence 314/478
Processing sentence 315/478
Processing sentence 316/478
Processing sentence 317/478
Processing sentence 318/478
Processing sentence 319/478
Processing sentence 320/478
Processing sentence 321/478
Processing sentence 322/478
Processing sentence 323/478
Processing sentence 324/478
Processing sentence 325/478
Processing sentence 326/478
Processing sentence 327/478
Processing sentence 328/478
Processing sentence 329/478
Processing sentence 330/478
Processing sentence 331/478
Processing sentence 332/478
Pro